In [2]:
from selenium import webdriver
import selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import datetime as dt
from os import listdir
from os.path import isfile, join
import pandas as pd
import time
import os
import requests
import lxml.html as lh
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
from messages import *
from selenium.webdriver.chrome.options import Options

In [3]:
class kotak:
    def __exit__(self, exc_type, exc_value, traceback):
            self.driver.quit()
    
    def refresh(self):
        temp=Select(self.driver.find_element_by_css_selector('#watchlistCollection'))
        temp.select_by_visible_text('NIFTY')
        temp=Select(self.driver.find_element_by_css_selector('#watchlistCollection'))
        temp.select_by_visible_text('SENSEX')

    def Stock(self,index = 'NIFTY',strike = 11500,side='BUY',o_type='CE',qty=1,typ='Market',price=1,otyp='Normal',trigger='1',tp=False):
        qty=str(qty)
        price=str(price)
        self.driver.maximize_window()
        self.closeWindow()
        if otyp=='CO':
            otyp='Super Multiple'
        elif otyp=='Margin':
            otyp='Margin Trading'
        self.driver.find_element_by_css_selector('#btnPlaceOrder').click()
        while True:
            try:
                temp=Select(self.driver.find_element_by_css_selector('#orderBuySell'))
                break
            except:
                continue
        temp.select_by_visible_text(side)
        temp=self.driver.find_element_by_css_selector('#orderSymbolSearch')
        temp.send_keys(symbol)
        temp=self.driver.find_element_by_id('orderSymbolSearchWrap')
        uiid='ui-id-'+temp.get_attribute('innerHTML').split('ui-id-')[1].split('"')[0]
        for i in range(50):
            if self.driver.find_element_by_css_selector('#'+uiid).get_attribute('innerHTML').count('<li')>0:
                break
            time.sleep(0.1)
        if i == 10:
            print('symbol Not done')#################
        temp=self.driver.find_element_by_css_selector('#orderSymbolSearch')
        temp.send_keys(selenium.webdriver.common.keys.Keys.ENTER)
        for i in range(30):
            if not self.driver.find_element_by_id('orderLTP').text=='':
                break
            time.sleep(0.1)
        if i == 10:
            print('LTP Not done')#################
        temp=self.driver.find_element_by_css_selector('#orderQuantity')
        temp.send_keys(qty)
        temp=Select(self.driver.find_element_by_css_selector('#orderType'))
        #Limit,Market,SL,SL-M
        temp.select_by_visible_text(typ)
        #orderPrice
        if typ=='SL' or typ=='Limit':
            temp=self.driver.find_element_by_css_selector('#orderPrice')
            price=str(price)
            temp.send_keys(price)
        temp=Select(self.driver.find_element_by_css_selector('#orderProduct'))
        #Normal,Margin Trading,Super Multiple,Buy Delivery
        temp.select_by_visible_text(otyp)
        #orderTriggerPrice only for super multiple
        if otyp=='Super Multiple':
            temp=self.driver.find_element_by_css_selector('#orderTriggerPrice')
            trigger=str(trigger)
            temp.send_keys(trigger)
        time.sleep(0.5)
        temp=self.driver.find_element_by_css_selector('#orderQuantity')
        temp.clear()
        temp.send_keys(str(qty))
        self.driver.find_element_by_css_selector('#btnConfirmOrder').click()
        time.sleep(0.5)
        for i in range(5):
            try:
                self.driver.find_element_by_css_selector('#btnPostOrder').click()
                break
            except:
                print('retrying')
                time.sleep(0.5)
        self.book['Stock'].append(symbol)
        self.book['OLTP'].append(self.driver.find_element_by_id('orderLTP').text)
        self.book['OLTP'].append(False)
        self.book['TP'].append(tp)
        self.book['SL'].append(trigger)
        self.book['Placed'].append(True)
        self.book['Closed'].append(False)
        self.book['Otime'].append(dt.datetime.now())
        self.book['Ctime'].append(False)
        
        self.closeWindow()
        
    def StockClose(self,name='suzlon'):
        self.driver.maximize_window()
        self.closeWindow()
        self.driver.find_element_by_id('netPositionsTab').click()
        for i in range(10):
            try:
                temp=self.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l0.r0 > input')
                break
            except:
                time.sleep(0.5)
                print('retrying')

        for i in range(15):
            try:
                temp.send_keys(name)
                break
            except:
                time.sleep(0.5)
        try:
            for temp in self.driver.find_elements_by_class_name('squareoff'):
                temp.click()
                for i in range(10):
                    try:
                        temp=Select(self.driver.find_element_by_css_selector('#orderType'))
                        #Limit,Market,SL,SL-M
                        temp.select_by_visible_text('Market')
                        self.driver.find_element_by_id('btnConfirmOrder').click()
                        break
                    except:
                        time.sleep(0.5)
                        print('waiting for confirm button')

            print('done place order')
        except:
            print('Error closing trade')
        try:
            for i in range(10):
                try:
                    self.driver.find_element_by_id('btnPostOrder').click()
                    break
                except:
                    time.sleep(0.5)
                    print('waiting for confirm button')
            print('done')
        except:
            print('Error closing trade')
        self.closeWindow()
        index=self.book['Stock'].index(name.upper())
        self.book['Closed'][index]=True
        self.book['CLTP'][index]=False
        self.book['Ctime'].append(dt.datetime.now())
        
        
    def closeWindow(self):
        try:
            self.driver.find_element_by_css_selector('#popup1 > div > div > div.launchHeader.ui-draggable-handle > span.closelaunch').click()
        except:
            pass
    
    def __init__(self,user_name='Login_username',password_='Login_password'):
        self.book={'Index':[],'Strike':[],'Side':[],'Type':[],'OLTP':[],'CLTP':[],'TP':[],'SL':[],'Placed':[],'Closed':[],'Otime':[],'Ctime':[]}
        url='https://mtrade.kotaksecurities.com/TSTerminal/Platform/Account/Login'
        driver = webdriver.Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        username=self.driver.find_element_by_name('username')
        username.send_keys(user_name)
        username=self.driver.find_element_by_name('password')
        username.send_keys(password_)
        self.driver.find_element_by_css_selector('#btnSecurityKey').click()
        key=input('Enter OTP: ')
        username=self.driver.find_element_by_name('securitykey')
        username.send_keys(key)
        self.driver.maximize_window()
        self.driver.find_element_by_css_selector('#btnSignIn').click()
        print('Going to terminal...')
        time.sleep(5)
        for i in range(40):
            try:
                self.driver.find_element_by_css_selector('#btnPlaceOrder').click()
                self.closeWindow()
                break
            except:
                print(i,'#####')
                time.sleep(1.5)

In [4]:
temp=kotak(user_name='Login_username',password_='Login_password')

Enter OTP: 2172
Going to terminal...


In [113]:
def Option(exchange='NFO',segment='OPTIDX',side='BUY',o_type='CE',strike='11500',oltp=None,qty=1):
    try:
        temp.driver.find_element_by_css_selector('#btnPlaceOrder').click()
    except:
        temp.closeWindow()
        temp.driver.find_element_by_css_selector('#btnPlaceOrder').click()
    while True:
        try:
            exchange_select=Select(temp.driver.find_element_by_id('exchangeSegment'))
            break
        except:
            continue
    exchange_select.select_by_visible_text(exchange)
    while True:
        try:
            segment_select=Select(temp.driver.find_element_by_id('segmentInstruments'))
            break
        except:
            continue
    segment_select.select_by_visible_text(segment)
    while True:
        try:
            side_select=Select(temp.driver.find_element_by_id('orderBuySell'))
            break
        except:
            continue
    side_select.select_by_visible_text(side)
    symbol_field=temp.driver.find_element_by_css_selector('#orderSymbolSearch')
    symbol_field.clear()
    symbol_field.send_keys('NIFTY')
    symbol_field=temp.driver.find_element_by_id('orderSymbolSearchWrap')
    uiid='ui-id-'+symbol_field.get_attribute('innerHTML').split('ui-id-')[1].split('"')[0]
    for i in range(10):
        if temp.driver.find_element_by_css_selector('#'+uiid).get_attribute('innerHTML').count('<li')>0:
            break
        time.sleep(0.5)
    if i == 10:
        print('symbol Not done')#################
    symbol_field=temp.driver.find_element_by_css_selector('#orderSymbolSearch')
    if o_type=='PE':
        symbol_field.send_keys(selenium.webdriver.common.keys.Keys.DOWN)
        symbol_field.send_keys(selenium.webdriver.common.keys.Keys.ENTER)
    else:
        symbol_field.send_keys(selenium.webdriver.common.keys.Keys.ENTER)

    while True:
        try:
            strike_select=Select(temp.driver.find_element_by_id('orderStrikePrice'))
            break
        except:
            continue
            
    while True:
        try:
            strike_select.select_by_visible_text(str(strike))
            break
        except:
            continue    
    if not oltp == None:
        OLTP = temp.driver.find_element_by_id('orderPrice')
        OLTP.clear()
        OLTP.send_keys(str(oltp))
    qty_select= temp.driver.find_element_by_id('orderQuantity')
    qty_select.clear()
    qty_select.send_keys(str(qty))
    time.sleep(0.5)
    temp.driver.find_element_by_css_selector('#btnConfirmOrder').click()
    time.sleep(0.5)
    for i in range(5):
        try:
            temp.driver.find_element_by_css_selector('#btnPostOrder').click()
            break
        except:
            print('retrying')
            time.sleep(0.5)
    temp.closeWindow()
    temp.refresh()

In [116]:
t1=dt.datetime.now()
OptionClose()
t2=dt.datetime.now()
t2-t1

done place order
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
waiting for confirm button
done


datetime.timedelta(0, 8, 455876)

In [98]:
security='NIFTY'
exchange='NFO'
side='BUY'
qty=1

In [109]:
def OptionClose(security='NIFTY',exchange='NFO',side='BUY',qty='1'):
    temp.driver.find_element_by_id('netPositionsTab').click()
    try:
        temp.driver.maximize_window()
    except:
        pass
    while True:
        try:
            security_select=temp.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l0.r0 > input')
            break
        except:
            pass
    while True:
        try:
            security_select=temp.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l0.r0 > input')
            break
        except:
            continue
    security_select.send_keys(security)

    security_select=temp.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l0.r0 > input')
    while True:
        try:
            security_select=temp.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l1.r1 > input')
            break
        except:
            continue
    security_select.send_keys(exchange)
    if side == 'BUY':
        while True:
            try:
                security_select=temp.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l4.r4 > input')
                break
            except:
                continue
        security_select.send_keys(str(qty))
    else:
        while True:
            try:
                security_select=temp.driver.find_element_by_css_selector('#orderBookGrid > div.slick-headerrow.ui-state-default > div.slick-headerrow-columns > div.ui-state-default.slick-headerrow-column.l6.r6 > input')
                break
            except:
                continue
        security_select.send_keys(str(qty))

    try:
        for action in temp.driver.find_elements_by_class_name('squareoff'):
            action.click()
            for i in range(10):
                try:
                    action=Select(temp.driver.find_element_by_css_selector('#orderType'))
                    #Limit,Market,SL,SL-M
                    action.select_by_visible_text('Market')
                    temp.driver.find_element_by_id('btnConfirmOrder').click()
                    break
                except:
                    time.sleep(0.5)
                    print('waiting for confirm button')

        print('done place order')
    except:
        print('Error closing trade')
    try:
        for i in range(10):
            try:
                temp.driver.find_element_by_id('btnPostOrder').click()
                break
            except:
                time.sleep(0.5)
                print('waiting for confirm button')
        print('done')
    except:
        print('Error closing trade')
    temp.closeWindow()
    temp.refresh()

In [107]:
temp.refresh()